In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58889256/58889256 [==============================] - 10s 0us/step


In [2]:
# Freeze base model
base_model.trainable = False

In [68]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(7, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [55]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [69]:
model.compile(loss = 'categorical_crossentropy' , optimizer= "SGD" , metrics = ['accuracy'])

In [70]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
)

In [71]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('Dataset1/Train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('Dataset1/Valid', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1063 images belonging to 7 classes.
Found 329 images belonging to 7 classes.


In [72]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

Epoch 1/10
33/33 [==============================] - 156s 5s/step - loss: 5.8268 - accuracy: 0.6134 - val_loss: 0.6453 - val_accuracy: 0.8541
Epoch 2/10
33/33 [==============================] - 164s 5s/step - loss: 0.5229 - accuracy: 0.8909 - val_loss: 0.2482 - val_accuracy: 0.9271
Epoch 3/10
33/33 [==============================] - 161s 5s/step - loss: 0.3282 - accuracy: 0.9276 - val_loss: 0.5820 - val_accuracy: 0.8754
Epoch 4/10
33/33 [==============================] - 160s 5s/step - loss: 0.1910 - accuracy: 0.9567 - val_loss: 0.2636 - val_accuracy: 0.9271
Epoch 5/10
33/33 [==============================] - 156s 5s/step - loss: 0.1892 - accuracy: 0.9558 - val_loss: 0.1043 - val_accuracy: 0.9726
Epoch 6/10
33/33 [==============================] - 158s 5s/step - loss: 0.1204 - accuracy: 0.9727 - val_loss: 0.1462 - val_accuracy: 0.9483
Epoch 7/10
33/33 [==============================] - 158s 5s/step - loss: 0.0739 - accuracy: 0.9746 - val_loss: 0.1065 - val_accuracy: 0.9696
Epoch 8/10
33

In [73]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

10/10 [==============================] - 35s 3s/step - loss: 0.1474 - accuracy: 0.9544


[0.1474313884973526, 0.954407274723053]